# 🔒 AES + LSB Steganography – Message Hiding (Hard Level - English)

In [1]:
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

def aes_encrypt(message):
    key = get_random_bytes(32)
    iv = get_random_bytes(12)
    cipher = AES.new(key, AES.MODE_GCM, nonce=iv)
    ciphertext, tag = cipher.encrypt_and_digest(message.encode('utf-8'))
    encrypted_data = iv + tag + ciphertext
    return key, encrypted_data

def hide_aes_data(image_path, data, output_name="C_encoded_aes.png"):
    binary_data = ''.join(format(b, '08b') for b in data) + '1111111111111111'
    img = Image.open(image_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    encoded = img.copy()
    data_idx = 0
    for y in range(encoded.size[1]):
        for x in range(encoded.size[0]):
            r, g, b = encoded.getpixel((x, y))
            rgb = [r, g, b]
            for i in range(3):
                if data_idx < len(binary_data):
                    rgb[i] = (rgb[i] & ~1) | int(binary_data[data_idx])
                    data_idx += 1
            encoded.putpixel((x, y), tuple(rgb))
            if data_idx >= len(binary_data):
                encoded.save(output_name)
                print(f"[✓] Encrypted message successfully hidden in: {output_name}")
                return

In [ ]:
# Example usage
message = "Bybit attack has been initiated."
key, encrypted = aes_encrypt(message)
hide_aes_data("C.png", encrypted)
print("AES key (hex):", key.hex())

# 🔓 AES + LSB Steganography – Message Revealing (Hard Level - English)

In [3]:
from PIL import Image
from Crypto.Cipher import AES

def reveal_aes_data(image_path, key_hex):
    key = bytes.fromhex(key_hex)
    img = Image.open(image_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    binary_data = ""
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            for color in img.getpixel((x, y)):
                binary_data += str(color & 1)
    byte_data = []
    for i in range(0, len(binary_data), 8):
        byte = binary_data[i:i+8]
        if byte == '11111111' and binary_data[i+8:i+16] == '11111111':
            break
        byte_data.append(int(byte, 2))
    data = bytes(byte_data)
    iv, tag, ciphertext = data[:12], data[12:28], data[28:]
    cipher = AES.new(key, AES.MODE_GCM, nonce=iv)
    try:
        return cipher.decrypt_and_verify(ciphertext, tag).decode('utf-8')
    except Exception:
        return "[❌] Incorrect key or corrupted data."

In [ ]:
print(reveal_aes_data("C_encoded_aes.png", "516b98222ab435542273e10f17a9f6c8105358abd75e7bc0e3d02d21b31207b0"))

In [ ]:
# Example usage
# print(reveal_aes_data("C_encoded_aes.png", "your_hex_key_here"))